In [1]:
try:
    from elasticsearch import Elasticsearch
    import pandas as pd
    import json 
    from elasticsearch import helpers 
    import numpy as np
    from sklearn.cluster import KMeans
    from sklearn.metrics import mean_squared_error
    import warnings
    warnings.filterwarnings("ignore")
    print("Loaded.....")    
except Exception as e: 
    print("Some modules are missing{}".format(e))

Loaded.....


In [2]:
movies=pd.read_csv("movies.csv")
ratings=pd.read_csv("ratings.csv")

In [3]:
ENDPOINT= "http://localhost:9200/"
es=Elasticsearch(timeout=600,hosts=ENDPOINT)

# SPLITTING GENRES 

In [4]:
movies=movies.join(movies.pop('genres').str.get_dummies("|"))
movies.replace(0, np.nan,inplace=True)
df=pd.merge(movies,ratings,on="movieId", how="outer")

# Calculation means per user and genre

In [5]:
genres=movies.columns[2:]


In [9]:
def f(x):
    y = x.dropna()
    return np.nan if y.empty else y

means_list=[]
genre_rating=pd.DataFrame()
for i in genres:
    
    tempdf=df.groupby(['userId',i] )['rating'].mean().agg(f).to_frame(name =i+" average")
    means_list.append(tempdf)
    means=means_list[0]
for i in range(1,len(means_list)):
    means=pd.merge(means,means_list[i],on="userId", how="outer")
means=means.sort_values(by=['userId'])
means.drop('(no genres listed) average',axis='columns', inplace=True)

,Action average,Adventure average,Animation average,Children average,Comedy average,Crime average,Documentary average,Drama average,Fantasy average,Film-Noir average,Horror average,IMAX average,Musical average,Mystery average,Romance average,Sci-Fi average,Thriller average,War average,Western average
userId,,,,,,,,,,,,,,,,,,,
1.0,2.800000,2.166667,2.000000,2.500000,2.000000,2.500000,NaN,2.571429,2.125000,NaN,3.000000,NaN,3.000000,NaN,3.500000,2.400000,2.833333,2.000000,3.0
2.0,3.315789,3.750000,3.666667,3.500000,3.093750,3.666667,NaN,3.615385,3.400000,NaN,3.500000,4.000,3.750000,4.000000,3.590909,3.800000,3.333333,3.800000,5.0
3.0,3.468750,3.300000,2.875000,2.700000,3.611111,3.700000,3.666667,3.920000,3.000000,NaN,3.375000,2.875,2.500000,3.750000,3.650000,3.142857,3.545455,4.166667,4.0
4.0,4.274194,4.338710,4.708333,4.634146,4.329545,4.157895,4.000000,4.464286,4.567568,5.0,3.944444,5.000,4.833333,4.090909,4.500000,4.255814,4.022727,4.750000,3.0
5.0,4.000000,3.825000,3.900000,3.892857,3.981818,3.450000,3.666667,3.807692,3.750000,NaN,3.875000,4.000,4.250000,3.333333,4.081081,4.000000,3.625000,4.000000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667.0,3.266667,3.571429,4.000000,4.000000,3.875000,3.666667,NaN,3.707317,4.000000,NaN,NaN,4.000,3.666667,4.000000,3.260870,4.250000,3.500000,3.833333,3.0
668.0,3.500000,3.000000,3.000000,3.000000,3.444444,4.571429,NaN,4.307692,4.000000,NaN,5.000000,NaN,NaN,3.000000,5.000000,NaN,4.500000,4.500000,NaN
669.0,3.777778,3.800000,NaN,NaN,3.400000,3.750000,NaN,3.416667,3.000000,5.0,2.571429,NaN,NaN,3.666667,3.250000,2.875000,3.076923,NaN,5.0


In [8]:
data4kmeans=means.fillna(0)

# CLUSTERING 

In [43]:
X=data4kmeans[['Action average','Adventure average','Animation average','Children average','Comedy average','Crime average','Documentary average','Drama average','Fantasy average','Film-Noir average','Horror average','IMAX average','Musical average','Mystery average','Romance average','Sci-Fi average','Thriller average','War average','Western average']].values

In [44]:
kmeans = KMeans(n_clusters=8)
predictions = kmeans.fit_predict(X)

In [45]:
n_clusters=8
clusters = [X[predictions== i] for i in range(n_clusters)]

centroids = kmeans.cluster_centers_
ClusterMeans=pd.DataFrame(columns=['userId','Action average','Adventure average','Animation average','Children average','Comedy average','Crime average','Documentary average','Drama average','Fantasy average','Film-Noir average','Horror average','IMAX average','Musical average','Mystery average','Romance average','Sci-Fi average','Thriller average','War average','Western average'])
myUserId=1
for i in predictions:
    cluster_number=i
    myValues=centroids[cluster_number]
    new_row = {'userId':myUserId,'Action average':myValues[0],'Adventure average':myValues[1],'Animation average':myValues[2],'Children average':myValues[3],'Comedy average':myValues[4],'Crime average':myValues[5],'Documentary average':myValues[6],'Drama average':myValues[7],'Fantasy average':myValues[8],'Film-Noir average':myValues[9],'Horror average':myValues[10],'IMAX average':myValues[11],'Musical average':myValues[12],'Mystery average':myValues[13],'Romance average':myValues[14],'Sci-Fi average':myValues[15],'Thriller average':myValues[16],'War average':myValues[17],'Western average':myValues[18]}
    ClusterMeans = ClusterMeans.append(new_row, ignore_index=True)
    myUserId+=1
ClusterMeans.set_index('userId', inplace=True)

In [66]:
filleddata=data4kmeans.where(data4kmeans!=0,ClusterMeans)
filleddata

,Action average,Adventure average,Animation average,Children average,Comedy average,Crime average,Documentary average,Drama average,Fantasy average,Film-Noir average,Horror average,IMAX average,Musical average,Mystery average,Romance average,Sci-Fi average,Thriller average,War average,Western average
userId,,,,,,,,,,,,,,,,,,,
1.0,2.800000,2.166667,2.000000,2.500000,2.000000,2.500000,0.410232,2.571429,2.125000,0.297945,3.000000,0.857923,3.000000,2.739775,3.500000,2.400000,2.833333,2.000000,3.000000
2.0,3.315789,3.750000,3.666667,3.500000,3.093750,3.666667,0.038760,3.615385,3.400000,0.038760,3.500000,4.000000,3.750000,4.000000,3.590909,3.800000,3.333333,3.800000,5.000000
3.0,3.468750,3.300000,2.875000,2.700000,3.611111,3.700000,3.666667,3.920000,3.000000,0.122059,3.375000,2.875000,2.500000,3.750000,3.650000,3.142857,3.545455,4.166667,4.000000
4.0,4.274194,4.338710,4.708333,4.634146,4.329545,4.157895,4.000000,4.464286,4.567568,5.000000,3.944444,5.000000,4.833333,4.090909,4.500000,4.255814,4.022727,4.750000,3.000000
5.0,4.000000,3.825000,3.900000,3.892857,3.981818,3.450000,3.666667,3.807692,3.750000,0.122059,3.875000,4.000000,4.250000,3.333333,4.081081,4.000000,3.625000,4.000000,2.997251
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667.0,3.266667,3.571429,4.000000,4.000000,3.875000,3.666667,0.038760,3.707317,4.000000,0.038760,3.218069,4.000000,3.666667,4.000000,3.260870,4.250000,3.500000,3.833333,3.000000
668.0,3.500000,3.000000,3.000000,3.000000,3.444444,4.571429,0.201807,4.307692,4.000000,0.126506,5.000000,2.101009,2.382523,3.000000,5.000000,3.854501,4.500000,4.500000,0.132530
669.0,3.777778,3.800000,0.612916,1.137437,3.400000,3.750000,0.410232,3.416667,3.000000,5.000000,2.571429,0.857923,0.767449,3.666667,3.250000,2.875000,3.076923,2.452609,5.000000


# Data for new metric

In [47]:
avgratings=ratings.groupby(['movieId'])['rating'].mean()
avgratings

movieId
1         3.872470
2         3.401869
3         3.161017
4         2.384615
5         3.267857
            ...   
161944    5.000000
162376    4.500000
162542    5.000000
162672    3.000000
163949    5.000000
Name: rating, Length: 9066, dtype: float64

In [48]:
movies1=pd.read_csv("movies.csv")

ratings1=pd.merge(movies1,avgratings,on="movieId",how="outer")


In [49]:
n=pd.merge(ratings1,movies,on="movieId",how="outer")
n=n.drop(columns="title_y")
n=n.drop(columns="(no genres listed)")


In [50]:
n=n.rename(columns={'Action':'Action average','Adventure':'Adventure average','Animation':'Animation average','Children':'Children average','Comedy':'Comedy average','Crime':'Crime average','Documentary':'Documentary average','Drama':'Drama average','Fantasy':'Fantasy average','Film-Noir':'Film-Noir average','Horror':'Horror average','IMAX':'IMAX average','Musical':'Musical average','Mystery':'Mystery average','Romance':'Romance average','Sci-Fi':'Sci-Fi average','Thriller':'Thriller average','War':'War average','Western':'Western average'})



In [51]:
which_user=int(input("Tell me your user Id"))

Tell me your user Id13


In [52]:
m=ratings[(ratings["userId"]==which_user)]
m.rename(columns={'rating': 'user_rating'}, inplace=True)
m.drop("timestamp",axis=1,inplace=True)
m.drop("userId",axis=1,inplace=True)


In [53]:
mn=pd.merge(n,m,on="movieId",how="outer")
mn=mn.fillna(0)
r=filleddata.loc[which_user,:]
mn=mn.where(mn!=1.0,r ,axis=1)
mn.iloc[0, mn.columns.get_loc('movieId')] = 1.0
dfend=mn



In [54]:
dfend.drop("title_x",axis=1)
dfend.drop("genres",axis=1)
dfend.drop("user_rating",axis=1)
dfend.drop("rating",axis=1)
dfend.replace(0,np.nan,inplace=True)

dfend["ratingprediction"]=dfend[['Action average','Adventure average','Animation average','Children average','Comedy average','Crime average','Documentary average','Drama average','Fantasy average','Film-Noir average','Horror average','IMAX average','Musical average','Mystery average','Romance average','Sci-Fi average','Thriller average','War average','Western average']].mean(axis=1)

dfend=dfend.fillna(0)
dfend


,movieId,title_x,genres,rating,Action average,Adventure average,Animation average,Children average,Comedy average,Crime average,...,IMAX average,Musical average,Mystery average,Romance average,Sci-Fi average,Thriller average,War average,Western average,user_rating,ratingprediction
0,1.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.872470,0.00,3.85,4.0,3.884615,3.568182,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,5.0,3.824196
1,2.0,Jumanji (1995),Adventure|Children|Fantasy,3.401869,0.00,3.85,0.0,3.884615,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,3.850932
2,3.0,Grumpier Old Men (1995),Comedy|Romance,3.161017,0.00,0.00,0.0,0.000000,3.568182,0.0,...,0.0,0.0,0.0,3.9,0.000000,0.0,0.0,0.0,0.0,3.734091
3,4.0,Waiting to Exhale (1995),Comedy|Drama|Romance,2.384615,0.00,0.00,0.0,0.000000,3.568182,0.0,...,0.0,0.0,0.0,3.9,0.000000,0.0,0.0,0.0,0.0,3.810823
4,5.0,Father of the Bride Part II (1995),Comedy,3.267857,0.00,0.00,0.0,0.000000,3.568182,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,3.568182
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9120,162672.0,Mohenjo Daro (2016),Adventure|Drama|Romance,3.000000,0.00,3.85,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,3.9,0.000000,0.0,0.0,0.0,0.0,3.904762
9121,163056.0,Shin Godzilla (2016),Action|Adventure|Fantasy|Sci-Fi,0.000000,3.75,3.85,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,3.583333,0.0,0.0,0.0,0.0,3.750379
9122,163949.0,The Beatles: Eight Days a Week - The Touring Y...,Documentary,5.000000,0.00,0.00,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.038760
9123,164977.0,The Gay Desperado (1936),Comedy,0.000000,0.00,0.00,0.0,0.000000,3.568182,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,3.568182


In [55]:
data4m=dfend.to_dict("records")


def generator(data4m):
    for c,line in enumerate(data4m):
        yield{
         '_index':'data4m',
         '_type' :'_doc',
         '_id':line.get('movieId'),
         '_source':{
             'title':line.get('title_x',["No Data"]),
             'genres':line.get('genres',["No Data"]),
             'rating':line.get('rating',["No Data"]),
             'user_rating':line.get('user_rating',["No Data"]),
             'Action average':line.get('Action average',["No Data"]),
             'Adventure average':line.get('Adventure average',["No Data"]),
             'Animation average':line.get('Animation average',["No Data"]),
             'Children average':line.get('Children average',["No Data"]),
             'Comedy average':line.get('Comedy average',["No Data"]),
             'Crime average ':line.get('Crime average',["No Data"]),
             'Documentary average':line.get('Documentary average',["No Data"]),
             'Drama average':line.get('Drama average',["No Data"]),
             'Fantasy average':line.get('Fantasy average',["No Data"]),
             'Film-Noir average':line.get('Film-Noir average',["No Data"]),
             'Horror average':line.get('Horror average',["No Data"]),
             'IMAX average':line.get('IMAX average',["No Data"]),
             'Musical average':line.get('Musical average',["No Data"]),
             'Mystery average':line.get('Mystery average',["No Data"]),
             'Romance average':line.get('Romance average',["No Data"]),
             'Sci-Fi average':line.get('Sci-Fi average',["No Data"]),
             'Thriller average':line.get('Thriller average',["No Data"]),
             'War average':line.get('War average',["No Data"]),
             'Western average':line.get('Western average',["No Data"]),
             'ratingprediction':line.get('ratingprediction',["No Data"])
         }
             } 
    gen=generator(data4m)
   

In [56]:
 settings = {
    'settings': {
            'number_of_shards': 1,
            'number_of_replicas': 1,
            "similarity": {
             "custom_bm25": {
                "type": "BM25",
                "b": 0,
                "k1" : 0.9
                 
             }
          }
                
            }
    ,
    
    'mappings': {
        
            "properties": {
                "title": {
                    "type": "text",
                    "analyzer": "english"
                },
                "genres":{
                    "type": 'text',
                    "analyzer":"english"
                },
            
             "rating":{
                    "type": "short"
                
                    
             },
                          
          
             "user_rating":{
                    "type": "short"
                    
               }, 
            "ratingprediction":{
                "type":"short"
            }
            }
        
        
        
           }
    
} 

In [57]:
my=es.indices.create(index="dfend",ignore=[400,404],body=settings)

# Insert to ES

In [58]:
try:
    res=helpers.bulk(es,generator(data4m))
    print("Working")
except Exception as e:
    print("Done")

Working


In [59]:
user_request=input("What are you looking for?")
query_body = {
  "query": {
      "multi_match":{
          "query":user_request,
          "fields":["title" ,"genres"]
      }
  }
}
# Pass the query dictionary to the 'body' parameter of the
# client's Search() method, and have it return results:
result = es.search(index="data4m", body=query_body,size=10000)
    

What are you looking for?horro


# New Metric

In [60]:
def metric(BM25,rating,user_rating,ratingprediction):
        if (user_rating>0 and rating >0 ) :         
            return 0.25*(BM25+rating+user_rating +ratingprediction)
        elif (user_rating==0 or rating ==0 ): 
            return 0.33(BM25+rating+user_rating+ratingprediction)
        else :
            return BM25

In [61]:
 
for i in result["hits"]["hits"]:
    
    BM25_score=i["_score"]*0.5
    rating=i["_source"]["rating"]
    user_rating=i["_source"]["user_rating"]
    ratingprediction=i["_source"]["ratingprediction"]
    new_metric=metric(BM25_score,rating,user_rating,ratingprediction)
    i['new_score']=new_metric

result["hits"]["hits"]=sorted(result["hits"]["hits"],key=lambda i:i['new_score'],reverse=True )
    
print ("total hits:", len(result["hits"]["hits"]))

for i in result['hits']['hits']:
    print(i['_source']['title']+" "+"\n"+i['_source']['genres']+"\n")

total hits: 0
